In [2]:
!pip install pyarrow fastparquet

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\SeungWoo\\Anaconda3\\Lib\\site-packages\\~andas\\_libs\\algos.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer
from tqdm import tqdm
import os

# ==========================================
# 1. 로컬 Parquet 파일 로드
# ==========================================
# 다운로드 받은 파일명을 정확하게 입력해주세요.
# 예: train-00000-of-00055.parquet (파일명이 다르면 수정 필수!)
file_path = r"C:\Users\SeungWoo\Downloads/train-00000-of-00014.parquet"

print(f"📂 로컬 파일 로드 중: {file_path}")

try:
    # Parquet 파일 읽기
    raw_df = pd.read_parquet(file_path)
    print(f"✅ 로드 성공! 총 {len(raw_df)}개의 대화 세트 발견")
except Exception as e:
    print("❌ 파일 로드 실패. 경로와 파일명을 확인해주세요.")
    print(e)
    # 에러 발생 시 이후 코드 실행 중단을 위해 빈 DF 생성
    raw_df = pd.DataFrame()

# ==========================================
# 2. 한국어 데이터 필터링 & 구조 변환
# ==========================================
if not raw_df.empty:
    print("\n🧹 데이터 전처리 중 (한국어 추출 및 대화 풀기)...")
    
    data_list = []
    
    # 'language' 컬럼이 이미 존재합니다. 'ko'인 것만 필터링
    ko_df = raw_df[raw_df['language'] == 'ko']
    print(f"   └ 전체 데이터 중 한국어 대화 세트: {len(ko_df)}개 발견")

    # 대화 내용(Nested List) 풀기
    for _, row in tqdm(ko_df.iterrows(), total=len(ko_df)):
        conversation = row['conversation'] # [{'role':..., 'content':...}, ...] 형태
        model_name = row['model']
        
        # conversation 배열(리스트)을 순회
        # Parquet 로드 시 리스트가 numpy array일 수 있어 list로 변환 안전장치
        if hasattr(conversation, 'tolist'):
            conversation = conversation.tolist()
            
        for turn in conversation:
            role = turn.get('role')
            content = turn.get('content')
            
            if content: # 내용이 있는 경우만
                data_list.append({
                    'text': content,
                    'label': 'Human' if role == 'user' else 'AI',
                    'model': model_name if role == 'assistant' else 'Human'
                })

    df = pd.DataFrame(data_list)
    print(f"✅ 최종 분석용 데이터 준비 완료: 총 {len(df)}행 (Turn 단위)")

    # ==========================================
    # 3. 데이터 확인 및 시각화
    # ==========================================
    print("\n🔍 [상위 10개 샘플]")
    display(df.head(10))
    
    print("\n📊 [데이터 분포]")
    print(df['label'].value_counts())

    # ==========================================
    # 4. 토큰 길이 계산 (XLM-RoBERTa)
    # ==========================================
    model_ckpt = "xlm-roberta-base"
    tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
    print(f"\n🤖 토크나이저({model_ckpt}) 로드 완료")

    token_counts = []
    print("⏳ 토큰 길이 계산 중...")

    for text in tqdm(df['text']):
        # 텍스트가 문자열이 아닌 경우 대비
        text = str(text)
        tokens = tokenizer(text, add_special_tokens=True, truncation=False)['input_ids']
        token_counts.append(len(tokens))

    df['token_length'] = token_counts

    # ==========================================
    # 5. 시각화 (범위 제한 0~3000)
    # ==========================================
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(14, 6))

    # 5-1. 히스토그램
    plt.subplot(1, 2, 1)
    sns.histplot(df['token_length'], bins=50, kde=True, color='purple', binrange=(0, 3000))
    plt.xlim(0, 3000)
    plt.title('Token Length (Korean Subset)', fontsize=14, fontweight='bold')
    plt.xlabel('Token Count')

    # 5-2. Human vs AI 비교
    plt.subplot(1, 2, 2)
    sns.boxplot(x='label', y='token_length', data=df, palette="coolwarm")
    plt.ylim(0, 3000)
    plt.title('Length Comparison: Human vs AI', fontsize=14, fontweight='bold')

    plt.tight_layout()
    plt.show()

    print("\n📈 [요약 통계]")
    print(df.groupby('label')['token_length'].describe().round(1))

📂 로컬 파일 로드 중: C:/Users/SeungWoo/Downloads/train-00000-of-00014.parquet
❌ 파일 로드 실패. 경로와 파일명을 확인해주세요.
Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - No module named 'pandas.core.arrays._arrow_utils'
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.
